In [25]:
from paramiko import client
import paramiko
import os
host = "da0.eecs.utk.edu"
port = 443
username = "KelbakianiIrakli"
password = "ika"
command = "ls"

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

stdin, stdout, stderr = ssh.exec_command(command)
lines = stdout.readlines()
print(lines)
stdin, stdout, stderr = ssh.exec_command("echo 'R2R <artur@vaadin.com>' | ~/lookup/getValues A2a")
lines = stdout.readlines()
for i in lines:
    print(i.strip())
#print(lines)

['ForEachLine.sh\n', 'ForEachLine.txt\n', 'after_java.txt\n', 'after_py.txt\n', 'after_python.txt\n', 'arturVaadine.txt\n', 'before_java.txt\n', 'before_py.txt\n', 'contributors.txt\n', 'diff_java.txt\n', 'diff_py.txt\n', 'diff_python.txt\n', 'echo\n', 'java_comments.txt\n', 'lookup\n', 'oscar.py\n', 'python_comments.txt\n']
R2R <artur@vaadin.com>;Artur <artur@vaadin.com>;Artur Signell <artur.signell@itmill.com>;Artur Signell <artur@vaadin.com>;R2R <artur@vaadin.com>


In [26]:
contributors = {}
#contributors["jw <jw@squareup.com>"] = []
contributors["Kevin <kevin@github.com>"] = []
for key,value in contributors.items():
    stdin, stdout, stderr = ssh.exec_command("echo \"{0}\" | ~/lookup/getValues A2c".format(key))
    lines = stdout.readlines()[0].strip().split(";")[1:]
    contributors[key]= lines
    #print(len(lines[1:]))
    

In [27]:
print(contributors["Kevin <kevin@github.com>"])

['...']


In [35]:
direc= os.getcwd()
for key,value in contributors.items():
    for i in value:
        with open("processed_commit.txt", "a") as myfile:
            myfile.write(i +"\n")
        stdin, stdout, stderr = ssh.exec_command("echo {0} | ssh da4 ~/lookup/cmputeDiff3T.perl".format(i))
        res = stdout.readlines()
        for k in res:
            #print(k)
            print(k)
            if ".java" in k:
                with open("processed_java_commit.txt", "a") as myfile:
                    myfile.write(k +"\n")
                blobInfo = k.split(";")
                if len(blobInfo) == 3:
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> java_created_file.txt".format(blobInfo[-1]))
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("grep '[#|/|*]' java_created_file.txt -B 1 >> java_comments.txt")
                else:
                    print(blobInfo[-1], blobInfo[-2])
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> before_java.txt".format(blobInfo[-1]))
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> after_java.txt".format(blobInfo[-2]))
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("diff before_java.txt after_java.txt >> diff_java.txt")
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("grep -B 1 -A 1 '[#|/|*]' diff_java.txt >> java_comments.txt")
                with open("java_logs.txt", "a") as myfile:
                    myfile.write(k)
            if ".py" in k:
                blobInfo = k.split(";")
                with open("processed_python_commit.txt", "a") as myfile:
                    myfile.write(k +"\n")
                if len(blobInfo) == 3:
                    print("len is 3")
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> python_created_file.txt".format(blobInfo[-1]))
                    stdin, stdout, stderr = ssh.exec_command("grep -A 1 -B 1 '[#|/|*]' python_created_file.txt  >> python_comments.txt")
                else:
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> before_python.txt".format(blobInfo[-1]))
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> after_python.txt".format(blobInfo[-2]))
                    stdin, stdout, stderr = ssh.exec_command("diff before_python.txt after_python.txt >> diff_python.txt")
                    stdin, stdout, stderr = ssh.exec_command("grep -A 1 -B 1  '[#|/|*]' diff_python.txt -B 1 >> python_comments.txt")
                    stdin, stdout, stderr = ssh.exec_command("> before_java.txt; > after_java.txt; > diff_java.txt")
                with open("python_logs.txt", "a") as myfile:
                    myfile.write(k)

00023709d3a3652aafb96bcfb4803c735bed98df;/build/package.json;6ef5d86ae0ae7b860da90ab0feaff2e8f29273dc;ea5b8e6deea9145406b27bb80fc828a358fafa90

00027e892c8cc977727f86e5ea6b4bb9de8d9800;/package.json;b808ca77351e88fe0c79bd8b15cc7f193f8f1bf5;9a14dc86faa31ef61421a06aa30800d9dce43982

0003277836bc9e5af68bbe5c4f576402c5a143c1;/fragment.go;caaae9a196edd1ae8f802a9f197f3d1725f0551d;2fd54d5ff8d10e4d7dc94f213d714a19deb5139d

0004750a855c51e1fde71a4dbd6a4ef128d6b60b;/package.json;806bbbd3b2d7635b72678f5682d602f55ed43c5c;c9b92217f17c821d75dcdc66bb0b5c15a44dcebd

0008a26e0f3847d96fe958e4500d162772d47ecb;/package.json;a9fc0716ba25ab8ec4815462e042d53273a5dd75;d33590d7a1f7094dc0ecacfc024cad4673cfa67c

0008bc5f8209738ae5a4f82ec30298c12f35530b;/docs-translations/ko-KR/tutorial/debugging-main-process-vscode.md;60e0dd3ed0d3753b07b2a4a88ddc0cb34165e1d6;adab50234ef8a9f3588f1c01ddb88e4bbf5b51d4

0008bc5f8209738ae5a4f82ec30298c12f35530b;/docs-translations/ko-KR/api/net.md;ac6b01e6f6137888efd9125668a98e68bf9fa

001d7fc7eaaa8f6159c3b4f24201983888962fd2;/src/main/java/com/github/mobile/gauges/ui/GaugeViewActivity.java;3ec749ec7c2fc3187054681ff759fea8e3cdb145;c7bde0628e2586a6b3d0034ce8b83e1fb67a83d2

c7bde0628e2586a6b3d0034ce8b83e1fb67a83d2
 3ec749ec7c2fc3187054681ff759fea8e3cdb145
['c7bde0628e2586a6b3d0034ce8b83e1fb67a83d2\n']
[]
[]
001dcad3e240d0bb653e4d90fb9ab19db83595be;/src/test.coffee;72e459d2b8aad021576d54f2372fe750d3702864;fa4c6d6454407ed3f7eea047ae61bacf3f51fe78

001ee4c03c393fcaa87ac97bfc1bd72ddb4f6c46;/mirror/window.js;59237089e1873a8f106429655523940403cd960d;e911707cef0d87cdfff589e01f4f34cc1bee77d1

001ee4c03c393fcaa87ac97bfc1bd72ddb4f6c46;/mirror/package.json;0cea319af4d8dafec47eb0e9bdd66c2d10a16b6d;4c16a92843a2558b030285b107a18a89169477b6

001ee4c03c393fcaa87ac97bfc1bd72ddb4f6c46;/hash/package.json;71e347d12cb00a84619cf1983a0b558aaaaa056d;dda321474276eefdb1e579b91a2d36057dd7ce18

001ee4c03c393fcaa87ac97bfc1bd72ddb4f6c46;/prices/package.json;941357be325378bb6aa54643b3c47ffdf2fb2ea8;

0033c9659f36f38db7179959e810cfac39434e83;/spec/spec-helper.coffee;db907b685d7204457d12511d71e475ed9afb76f4;5fb160e87f4c0bc63ca56a7458ce2ec667ddfa27

0033c9659f36f38db7179959e810cfac39434e83;/src/config.coffee;c50384e18d0c546295aa0b4432f926346baef622;05bdc19a4f2e67ab3822bcd4ccca345e5047cb71

0033c9659f36f38db7179959e810cfac39434e83;/src/atom.coffee;96501496b6719c96a0fbbbf4f5469e8011b3766a;7a2982101f2703b7e9286d71ebc94f6cec9ad289

003522d6f714cf38da38a15077ff8a87f046aba3;/docs/development/build-instructions-osx.md;bca4dd61090e02c6309442501c29b07d51af348a;8d87a49881839f36fb7aa51302c4974c2dcdf749

00373cd3863951031999eea694d45419d94ab859;/exports/atom-api.coffee;c2294dd0f64fcaaa3721c525fee25e0e929555e1;77e8c8e907cd23a627c8d95386d2d254197883a8

0037af51f37a152a8814b68910c55ea9404d461b;/electron.gyp;fba2217146d6dafde3d8ce4237222ebc993390a2;971d3b47eb59666851d03b6b446392bac158b6d8

0037af51f37a152a8814b68910c55ea9404d461b;/package.json;32c763cf8767bd666c7cd0a0adf9ae8e6c38c690;d237b46970d9b71b

006c77a00fb26572592af51e5201fc7a602e1e3c;/lib/browser/init.js;d6a18361d69d6c8ca05f363d5885230a2e08fb46;4769faee0a39501ff4ecb044dcedb7be4c99377e

006c77a00fb26572592af51e5201fc7a602e1e3c;/lib/browser/api/exports/electron.js;ea9dd6eb5a2681278298f64e0d6634caa3e1fec6;7f97fcdbc4cf5653b193785b96ec8af6f7d96773

006c77a00fb26572592af51e5201fc7a602e1e3c;/lib/renderer/init.js;334aebda9cfe13dcb3cde4bb3c5ccee95fc050a1;166e64237de482f80eacb7dd52915584160a6f3b

006c77a00fb26572592af51e5201fc7a602e1e3c;/lib/renderer/api/remote.js;42435d8304c6fe8245ed95c705bf5906825bd88a;07aa247829352e245369767fb51152b0cfba3141

006c77a00fb26572592af51e5201fc7a602e1e3c;/lib/renderer/api/exports/electron.js;34a498fcc0653b4f4dfae449638c9ed5a7691cb5;3f0d3254cb8b02fb2c0e722cce63daf976edd522

006c77a00fb26572592af51e5201fc7a602e1e3c;/atom/common/node_bindings.cc;608cc94b09a0fde713ca17f701e738403fa0dacf;69e7906ffbbbdd7ce62235f2c9be0c7730eb5964

006fd61e5ca6223d4c0f14885c97d1c5e8ba9567;/src/login.coffee;f85c3b2b0193de12fb4ad

KeyboardInterrupt: 

In [ ]:
direc= os.getcwd()
for key,value in contributors.items():
    for i in value:
        stdin, stdout, stderr = ssh.exec_command("echo {0} | ssh da4 ~/lookup/cmputeDiff3T.perl".format(i))
        res = stdout.readlines()
        for k in res:
            print(k)
            if ".java" in k:
                print("java", end='')
                blobInfo = k.split(";")
                if len(blobInfo) == 3:
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> java_created_file.txt".format(blobInfo[-1]))
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("grep '[#|/|*]' java_created_file.txt -B 1 >> java_comments.txt")
                else:
                    print(blobInfo[-1], blobInfo[-2])
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> before_java.txt".format(blobInfo[-1]))
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> after_java.txt".format(blobInfo[-2]))
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("diff before_java.txt after_java.txt >> diff_java.txt")
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("grep '[#|/|*]' diff_java.txt -B 1 >> java_comments.txt")
                #stdin, stdout, stderr = ssh.exec_command("> before_java.txt; > after_java.txt; > diff_java.txt")
                
                #ftp_client=ssh.open_sftp()
                #ftp_client.get('/home/KelbakianiIrakli/diff.txt',localpath='abc.txt')
                #ftp_client.close()
                #sftp = ssh.open_sftp()
                #localpath = direc + '\\abc.txt'
                #remotepath = '/home/KelbakianiIrakli/diff.txt'
                #sftp.put(remotepath, localpath)
                #res = stdout.readlines()
                #print(res)
            elif ".py" in k:
                print("python", end='')
                blobInfo = k.split(";")
                if len(blobInfo) == 3:
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> python_created_file.txt".format(blobInfo[-1]))
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("grep '[#|/|*]' python_created_file.txt -B 1 >> python_comments.txt")
                else:
                    print(blobInfo[-1], blobInfo[-2])
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> before_python.txt".format(blobInfo[-1]))
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> after_python.txt".format(blobInfo[-2]))
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("diff before_java.txt after_python.txt >> diff_python.txt")
                    print(stdout.readlines())
                    stdin, stdout, stderr = ssh.exec_command("grep '[#|/|*]' diff_python.txt -B 1 >> python_comments.txt")
                #stdin, stdout, stderr = ssh.exec_command("> before_java.txt; > after_java.txt; > diff_java.txt")


In [2]:
array = []
localTxt = []

In [ ]:
with open('vaadin_contributors.txt', encoding="utf8") as f:
    lines = f.readlines()
for i in lines:
    array.append(i.strip())
print(array)

In [ ]:
for i in array:
    k = "echo \"{0}\" | ~/lookup/getValues a2A".format(i)
    print(k)
    stdin, stdout, stderr = ssh.exec_command(k)
    lines = stdout.readlines()
    aliasName = lines[0].split(";",1)[1]
    if aliasName not in AliasNames:
        AliasNames.append(aliasName.strip())

In [ ]:
print(AliasNames)

In [ ]:
newAliasNames = [x[:-1] for x in AliasNames]

In [ ]:
print(newAliasNames.remove("dependabot[bot] <49699333+dependabot[bot]@users.noreply.github.com>"))

In [ ]:
stdin, stdout, stderr = ssh.exec_command("echo 'dunand <jeffnadeau@gmail.com>' | ~/lookup/getValues A2c")
lines = stdout.readlines()[0].strip()
print(lines.split(";"))

In [ ]:
newAliasNames[3]

In [ ]:
userCommitDict = {}
for i in newAliasNames:
    print(i)
    stdin, stdout, stderr = ssh.exec_command("echo \"{0}\" | ~/lookup/getValues A2c".format(i))
    stdout = stdout.readlines()
    if(len(stdout) == 0):
        # This is necessary if user has 1 author name and no alias
        stdin, stdout, stderr = ssh.exec_command("echo \"{0}\" | ~/lookup/getValues a2c".format(i))
        stdout = stdout.readlines()
        if (len(stdout)== 0):
            print("Something wrong with the user: " + i)
            continue
    UserAndCommits = stdout[0].strip().split(";")
    userCommitDict[UserAndCommits[0]] = UserAndCommits[1:]

In [3]:
import pickle

#with open('filename.pickle', 'wb') as handle:
    #pickle.dump(userCommitDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('filename.pickle', 'rb') as handle:
    userCommitDict = pickle.load(handle)

#print(a == b)

In [4]:
stdin, stdout, stderr = ssh.exec_command("echo {0} | ssh da4 ~/lookup/cmputeDiff3T.perl".format(userCommitDict["Marlon <marlon@vaadin.com>"][0]))
stdout = stdout.readlines()
print(len(stdout))

2


In [ ]:
print(userCommitDict["Marlon <marlon@vaadin.com>"][0])

In [ ]:
stdin, stdout, stderr = ssh.exec_command("echo \"{0}\" | ~/lookup/getValues A2c".format("Teemu Suo-Anttila <teemusa@vaadin.com>"))
print(len(stdout.readlines()))
#print(stdout.readlines()[0].strip().split(";"))

In [19]:
direc= os.getcwd()
for key,value in userCommitDict.items():
    for i in value:
        stdin, stdout, stderr = ssh.exec_command("echo {0} | ssh da4 ~/lookup/cmputeDiff3T.perl".format(i))
        res = stdout.readlines()
        for k in res:
            #print(k)
            if ".java" in k:
                print("java", end='')
                blobInfo = k.split(";")
                print(blobInfo[-1], blobInfo[-2])
                stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> before_py.txt".format(blobInfo[-1]))
                stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> after_py.txt".format(blobInfo[-2]))
                stdin, stdout, stderr = ssh.exec_command("diff before.txt after.txt >> diff.txt")
                ftp_client=ssh.open_sftp()
                ftp_client.get('/home/KelbakianiIrakli/diff.txt',localpath='abc.txt')
                ftp_client.close()
                #sftp = ssh.open_sftp()
                #localpath = direc + '\\abc.txt'
                #remotepath = '/home/KelbakianiIrakli/diff.txt'
                #sftp.put(remotepath, localpath)
                #res = stdout.readlines()
                print(res)
            elif ".py" in k:
                blobInfo = k.split(";")
                blobInfo = k.split(";")
                print(blobInfo[-1], blobInfo[-2])
                stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> before_py.txt".format(blobInfo[-1]))
                stdin, stdout, stderr = ssh.exec_command("echo {0} | ~/lookup/showCnt blob >> after_py.txt".format(blobInfo[-2]))
                stdin, stdout, stderr = ssh.exec_command("diff before.txt after.txt >> diff_py.txt")
                print(blobInfo)
                print("python", end='')
                
            #print("-",end = '')
            #if ".py" in k:
                #print("Yes")





#for key,value in userCommitDict.items():
#    for val in value:
#        #print(val)
#        stdin, stdout, stderr = ssh.exec_command("echo {0} | ssh da4 ~/lookup/cmputeDiff3T.perl".format(val))
#        res = stdout.readlines()
#        print(res)

java9502401e5b2384963de6cde54e4989a26cd2e269
 337fb3d57aa2b861f33a06ca8e4e702dd6e6a1d1
['000c636be286fca5d8bc9bdaa7662047ae809ad5;/flow-server/src/test/java/com/vaadin/flow/shared/VaadinUriResolverTest.java;337fb3d57aa2b861f33a06ca8e4e702dd6e6a1d1;9502401e5b2384963de6cde54e4989a26cd2e269\n', '000c636be286fca5d8bc9bdaa7662047ae809ad5;/flow-server/src/test/java/com/vaadin/flow/server/MockServletServiceSessionSetup.java;75274eeb29be5712a0e2ae5b8687724a68408be0;59c998b79c7bc9bc80da5a0238e3e33c3a6be276\n', '000c636be286fca5d8bc9bdaa7662047ae809ad5;/flow-server/src/test/java/com/vaadin/flow/component/internal/HtmlDependencyParserTest.java;c9fbc6e24ac66ae8852dc53fb4bd182339c4e60b;056e668d7466b25ec01d8f9eef8e8ea1c90eeb1f\n', '000c636be286fca5d8bc9bdaa7662047ae809ad5;/flow-server/src/main/java/com/vaadin/flow/component/internal/HtmlDependencyParser.java;7c0e48e0025985e1c4bb7b02f76170a236b2c545;c49e9fac8499ddaed5d2e01a91b68d8a302ad6de\n']
java59c998b79c7bc9bc80da5a0238e3e33c3a6be276
 75274eeb29b

SFTPError: Garbage packet received

In [ ]:
for key in userCommitDict.keys():
    print(key)

In [ ]:

import threading
import time


class MyThread(threading.Thread):
    def run(self):                                         # Default called function with mythread.start()
        print("{} started!".format(self.getName()))        # "Thread-x started!"
        time.sleep(1)                                      # Pretend to work for a second
        print("{} finished!".format(self.getName()))       # "Thread-x finished!"

def main():
    for x in range(4):                                     # Four times...
        mythread = MyThread(name = "Thread-{}".format(x))  # ...Instantiate a thread and pass a unique ID to it
        mythread.start()                                   # ...Start the thread, run method will be invoked
        time.sleep(.9)                                     # ...Wait 0.9 seconds before starting another

if __name__ == '__main__':
    main()

In [ ]:
for key,value in userCommitDict.items():
    for i in value:
        stdin, stdout, stderr = ssh.exec_command("echo {0} | ssh da4 ~/lookup/cmputeDiff3T.perl".format(i))
        res = stdout.readlines()
        for k in res:
            print(k)
            #if ".java" in k:
                #print("java", end='')
            #elif ".py" in k:
                #print("python", end='')
            #print("-",end = '')
            #if ".py" in k:
                #print("Yes")
                